In [2]:
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
from glob import glob
from skimage.transform import resize
import sigpy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
from subtle.data_loaders import PreSlicedMPRLoader

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _create_example(fpath_slice, X, Y):
    fp = fpath_slice
    plane = fpath_slice.split('/')[-2]
    cnum = fpath_slice.split('/')[-3]
    
    example = tf.train.Example(
        features=tf.train.Features(feature={
            'X': _bytes_feature(X.tobytes()),
            'Y': _bytes_feature(Y.tobytes())
        })
    )
    return example.SerializeToString()

In [34]:
src_path = '/home/srivathsa/projects/studies/gad/mra_synth/preprocess/slices'
cases = sorted([c.split('/')[-1] for c in glob('{}/*'.format(src_path))])[:5]
data_list = ['{}/{}.npy'.format(src_path, cnum) for cnum in cases]
data_loader = PreSlicedMPRLoader(
    data_list, slice_axis=[0, 2 ,3], use_enh_mask=True, slices_per_input=7, enh_pfactor=1.75, shuffle=False,
    batch_size=1
)

In [35]:
len(data_loader.slice_list_files)

ax = [f for f in data_loader.slice_list_files if 'ax' in f]
sag = [f for f in data_loader.slice_list_files if 'sag' in f]
cor = [f for f in data_loader.slice_list_files if 'cor' in f]

print(len(ax), len(sag), len(cor))

500 624 272


In [47]:
from subtle.utils.experiment import get_experiment_data

exp_path = '../configs/experiments'
all_cases = get_experiment_data('mra_synth', dataset='all', dirpath_exp=exp_path)

guys = [c for c in all_cases if 'guys' in c.lower()]
guys = np.random.permutation(guys)

hh = [c for c in all_cases if 'hh' in c.lower()]
hh = np.random.permutation(hh)

iop = [c for c in all_cases if 'iop' in c.lower()]
iop = np.random.permutation(iop)

print(len(guys), len(hh), len(iop))

train = list(guys[:66]) + list(hh[:38]) + list(iop[:16])
val = list(guys[66:80]) + list(hh[38:46]) + list(iop[16:19])
test = [c for c in all_cases if c not in train and c not in val]

315 181 72


In [50]:
sorted(test)

['IXI002-Guys-0828',
 'IXI012-HH-1211',
 'IXI013-HH-1212',
 'IXI015-HH-1258',
 'IXI016-Guys-0697',
 'IXI019-Guys-0702',
 'IXI020-Guys-0700',
 'IXI022-Guys-0701',
 'IXI023-Guys-0699',
 'IXI026-Guys-0696',
 'IXI027-Guys-0710',
 'IXI028-Guys-1038',
 'IXI029-Guys-0829',
 'IXI030-Guys-0708',
 'IXI031-Guys-0797',
 'IXI034-HH-1260',
 'IXI035-IOP-0873',
 'IXI036-Guys-0736',
 'IXI037-Guys-0704',
 'IXI038-Guys-0729',
 'IXI039-HH-1261',
 'IXI040-Guys-0724',
 'IXI042-Guys-0725',
 'IXI043-Guys-0714',
 'IXI044-Guys-0712',
 'IXI045-Guys-0713',
 'IXI048-HH-1326',
 'IXI049-HH-1358',
 'IXI050-Guys-0711',
 'IXI052-HH-1343',
 'IXI053-Guys-0727',
 'IXI055-Guys-0730',
 'IXI056-HH-1327',
 'IXI058-Guys-0726',
 'IXI060-Guys-0709',
 'IXI062-Guys-0740',
 'IXI063-Guys-0742',
 'IXI064-Guys-0743',
 'IXI065-Guys-0744',
 'IXI066-Guys-0731',
 'IXI069-Guys-0769',
 'IXI070-Guys-0767',
 'IXI073-Guys-0755',
 'IXI074-Guys-0771',
 'IXI075-Guys-0754',
 'IXI078-Guys-0751',
 'IXI079-HH-1388',
 'IXI080-HH-1341',
 'IXI084-Guys-0

In [9]:
dl_len = data_loader.__len__()
tfr_path = '/mnt/raid/srivathsa/mra_synth/preprocess/test.tfrecord'
writer = tf.python_io.TFRecordWriter(tfr_path)

for idx in tqdm(np.arange(dl_len), total=dl_len):
    X, Y = data_loader.__getitem__(idx)
    print(X.shape, Y.shape)
    fpath_slice = data_loader.slice_list_files[idx]
    writer.write(_create_example(fpath_slice, X, Y))

/home/srivathsa/miniconda3/envs/py35gad/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3420 [00:00<?, ?it/s]

(1, 512, 512, 14) (1, 512, 512, 2)
(1, 512, 512, 14) (1, 512, 512, 2)


KeyboardInterrupt: 

In [ ]:
tfr = _create_example(fpath_slice, X, Y)